# UVW Explorations

A baseline vector is the vector from one antenna position to another antenna
position.  Baseline vectors and antenna positions can be represented in many
different coordinate frames.  For Earth based antennas, the antenna positions
can be represented in one of several terrestrial frames.  The difference between
two such antenna positions is a baseline vector in the same terrestrial frame.
To make maps of the sky from visibilities associated with baselines, the
baseline vectors must be represented in a celestial frame, often represented as
a *UVW frame*, which is a right handed coordinate frame with U pointing
eastward, V pointing northward, and W pointing to the phase center of the
observation.  This notebook explores the conversion from antenna positions in a
terrestrial frame to baseline vectors in a UVW frame.

## Background

One commonly used terrestrial frame is the International Terrestrial Reference
Frame (ITRF).  This is a right-handed XYZ frame with the origin at the center of
Earth, the Z axis passing through the north pole, the X axis passing through the
prime meridian, and the Y axis passing through longitude 90 degrees East.
Antenna positions represented in this frame are over 6 million meters from the
origin, which makes comparing antenna positions non-intuitive for humans, so
often the observatory's reference position will be subtracted from the ITRF
antenna positions.  This leads to a topocentric XYZ frame.  Conversion between
ITRF coordinates and the more familiar latitude, longitude, altitude (LLA)
representation can be done for a given ellipsoid model of Earth.  These days the
WGS84 ellipsoid is most commonly used.

Celestial positions are referred to the International Celestial Reference System
(ICRS) or the somewhat older "J2000" reference system.  These reference systems
are so close to each other (there is only a ~23 milliarcsecond rotation between
them) that they can often be used interchangeably.  Observing a celestial
position from a terrestrial position at a given time involves not only the
complex relationship between the celestial reference system and the terrestrial
reference system, but also the diurnal aberration seen by the observer and
various atmospheric effects such as refraction.  One useful reference for this
topic is the [SOFA Tools for Earth
Attitude](http://iausofa.org/2021_0512_C/sofa/sofa_pn_c.pdf) cookbook.

Calculation of baseline vectors in a UVW frame requires these inputs:

* Right ascension of phase center
* Declination of phase center
* Date and time of observation
* Terrestrial location of the observatory's reference location
* Antenna positions
* List of antenna pairs for which to compute baseline vectors
* Earth orientation parameters (EOP) from the International Earth Rotation
  Service (IERS) for the date of the observation.  The IERS publishes EOP values
  with a 1 day granularity, but often the time of the observation is used to
  interpolate between two (or more) daily values.

## Setup

We will use values from the header of an example UVH5 file included alongside
this notebook.  UVH5 is an HDF5 based file format for storing interferometric
data and associated metadata.  It is described in the [UVH5
Memo](https://github.com/RadioAstronomySoftwareGroup/pyuvdata/blob/main/docs/references/uvh5_memo.pdf).
To reduce the size of the example file, the visibility data have been removed.

First we will setup the Julia environment for the `UVWExplorer` package that
includes this notebook.  Because the `PyCASA` project is not (yet) registered
with the main Julia package registry we have to add it explicitly before
instantiating the rest of the environment.  This may take up a couple of minutes
the first time you run this notebook as it will download and precompile packages
from the Julia package registry.  Subsequent runs will take less time.

In [5]:
# Setup the environment
import Pkg
redirect_stderr(devnull) do
    Pkg.activate(dirname(@__DIR__))
    Pkg.add(url="https://github.com/david-macmahon/PyCASA.jl")
    Pkg.instantiate()
end

# Pull in UVWExplorer package
using UVWExplorer

## UVH5 metadata

Now we can use `UVWExplorer`'s `uvh5info` function to load relevant metadata about the
observation from the UVH5 file.  Here are a few things to note about the UVH5
metadata:

* Right ascension and declination are in radians
* Date/time is given as a Julian date
* Observatory position is given as geodetic latitude, longitude, and altitude
  with respect to the WGS84 ellipsoid.  Latitude and longitude are given in
  degrees, altitude in meters.  The coordinates shown below indicate that this
  file is from the Very Large Array (VLA) in New Mexico, USA.

In [6]:
o = uvh5info(joinpath(dirname(@__DIR__), "data/example_without_visdata.uvh5"))
@show o.ra o.dec o.jd o.obslla;

o.ra = 3.2686162621088966
o.dec = 0.03582094364245917
o.jd = 2.4598932634780095e6
o.obslla = (lat = 34.07861111111111, lon = -107.61777777777777, alt = 2124.0)


* Antenna positions are given in meters in a topocentric XYZ frame (i.e.
  relative to observatory position).  It is convenient to work with antenna
  positions in a `3xN` Matrix that can be left multiplied by a `3x3` rotation
  matrix.

In [7]:
o.antxyz

3×27 Matrix{Float64}:
  75.3561  -1407.46    -628.666   -39.8687   …   22.795   233.805   123.426
 489.407     -77.4865   -35.3721   -2.84079     148.519  -148.405   801.639
 721.544    -735.191   -327.651   -20.2104      220.001  -102.917  1182.12

* Baselines are specified by a Vector (i.e. list) of antenna index pairs.
Strictly speaking, UVH5 specified baselines by antenna numbers, but `uvh5info`
uses additional UVH5 metadata to convert the antenna numbers to antenna indexes
(i.e. column indexes into the antenna positions Matrix).  Only 13 of the VLA's
27 antennas were used in this observation and `uvh5info` excludes
autocorrelations.  This is why only 78 baselines (`12 * 13 / 2 == 78`) are
present.

In [8]:
o.bls

78-element Vector{Tuple{Int64, Int64}}:
 (22, 21)
 (22, 20)
 (22, 19)
 (22, 18)
 (22, 17)
 (22, 14)
 (22, 5)
 (22, 4)
 (22, 13)
 (22, 12)
 ⋮
 (4, 12)
 (4, 11)
 (4, 7)
 (13, 12)
 (13, 11)
 (13, 7)
 (12, 11)
 (12, 7)
 (11, 7)

## First look at UVW vectors

UVH5 files contain precomputed UVW values for each baseline at each time sample
(aka *integration*).  `uvh5info` returns info only for the first time sample.
Here is a sneak peek at these precomputed UVW values:

In [9]:
o.uvws

3×78 Matrix{Float64}:
   126.631  1375.06    1612.23    645.305  …  -131.353  152.978    284.332
  2085.22    369.18     280.199   541.572      319.919  -96.4425  -416.362
 -1569.8    -999.815  -1061.35   -733.635     -160.966  -11.7981   149.168

## Calculating UVW values

Calculating UVW is essentially expressing baseline vectors in a UVW frame.
A UVW frame has W pointing in the direction of the phase center (aka
*source*).  Relating the antenna positions to the "direction of the phase
center" involves a number of complicating factors depending on which frame is
used to calculate the direction.

* Proper motion (can be ignored for extragalactic radio sources)
* Parallax (can be ignored for extragalactic radio sources)
* Light deflection (or radio deflection) due to gravity of solar system bodies
  (primarily the Sun and Jupiter)
* Annual aberration due to Earth's orbital motion around the Sun
* Precession and nutation
* Earth orientation parameters (EOP)
* Diurnal aberration due to Earth rotation
* Diurnal parallax (can be ignored for extragalactic sources)
* Atmospheric refraction

Software that computes the phase center direction in a given frame, such as the
`atco13` function from the Essential Routines for Fundamental Astronomy (ERFA),
takes parameters that either specify the relevant values directly (e.g proper
motion) or allows computation of these values from related models (e.g.
refraction can be computed from atmospheric pressure, ambient temperature,
relative humidity, and wavelength).

For UVW calculations, we assume zero proper motion, zero parallax, and
zero refraction.  The first two are generally safe to ignore.  Refraction is a
bit trickier and will be discussed more later, but for now we will ignore it.

Of the remaining complicating factors, light deflection and aberration can be
derived from phase center direction and observatory position and velocity.
Observatory velocity can be derived from observatory position and time of
observation.  Earth orientation parameters cannot be modeled so they must be
provided explicitly if the software has no automated internal means of obtaining
this information.  The EOP parameters that are relevant here are `UT1-UTC`
(aka `DUT1`) given in milliseconds (of time) by IERS and the polar motion values
`xp` and `yp` given in arcseconds by IERS, but be sure to double check the units
returned from EOP software APIs and bew sure to convert as necessary.  For
example, most return `UT1-UTC` in seconds rather than the milliseconds.

This leads to the following generalized function for computing UVW values from
RA/Dec and other observational metadata:

```julia
radec2uvws(ra, dec, jd, obslla, antxyz, bls; dut1=0, xp=0, yp=0)
```

The parameters and there units are described in the following table:

| Parameter | Description            | Units             |
|----------:|:-----------------------|:------------------|
|        ra | Right ascention (ICRS) | radians           |
|       dec | Declination (ICRS)     | radians           |
|     jdutc | Julian Date (UTC)      | days              |
|    obslla | Observatory location   | LLA (see below)   |
|    antxyz | Antenna positions      | meters (ITRF/XYZ) |
|       bls | Baseline list          | antenna indices   |
|      dut1 | UTC-UT1                | seconds           |
|        xp | Polar motion X         | arcsec            |
|        yp | Polar motion Y         | arcsec            |

The `obslla` parameter can be any object that has `lat`, `lon`, and `alt`
fields, with `lat` and `lon` in degrees and `alt` in meters.  Our `o.obslla`
that we saw above can be used for this parameter.

The `antxyz` parameter is a 3xN matrix of antenna positions in meters in either
ITRF or topocentric XYZ frames.

The `bls` parameter is a vector of antenna index tuples as seen above.  This
allows calculation of a subset of all possible baselines.

The `UVWExplorer` package for Julia provides several modules that implement the
`radec2uvws` function using different techniques to compute UVW.  The modules
are:

* `TopoUVW` - Computes UVWs from topocentric/observed frame
* `GCRSUVW` - Computes UVWs from the Geocentric Celestial Reference System (GCRS)
* `CASAUVW` - Computes UVWs using CASA's `measures` class
* `PyUVW` - Computes UVWs from CGRS using `astropy`

The first two modules use functions from `ERFA.jl` (a Julia interface to
`libefra`) to implement UVW calculations.  The latter two use `PyCall.jl` to
call into Python implementations that perform the UVW calculations and return
the results.  Only the first one, `TopoUVW`, computes UVW from the
topocentric/observed frame.  The remaining ones compute UVW from the geocentric
frame (i.e. the GCRS). 

Let's see what they produce given the same metadata from the UVH5 file!

In [10]:
topouvws = TopoUVW.radec2uvws(o.ra, o.dec, o.jd, o.obslla, o.antxyz, o.bls)

3×78 Matrix{Float64}:
   126.631  1375.06    1612.23    645.305  …  -131.353  152.978    284.332
  2085.22    369.18     280.199   541.572      319.919  -96.4425  -416.362
 -1569.8    -999.815  -1061.35   -733.635     -160.966  -11.7981   149.168

In [11]:
gcrsuvws = GCRSUVW.radec2uvws(o.ra, o.dec, o.jd, o.obslla, o.antxyz, o.bls)

3×78 Matrix{Float64}:
   127.156  1375.15    1612.3     645.441  …  -131.273  152.954    284.227
  2085.19    368.834    279.793   541.41       319.952  -96.4811  -416.433
 -1569.8    -999.815  -1061.35   -733.635     -160.966  -11.7981   149.168

In [12]:
casauvws = CASAUVW.radec2uvws(o.ra, o.dec, o.jd, o.obslla, o.antxyz, o.bls)

3×78 Matrix{Float64}:
   127.161  1375.15    1612.31    645.442  …  -131.272  152.954    284.226
  2085.19    368.834    279.792   541.41       319.953  -96.4813  -416.434
 -1569.8    -999.814  -1061.35   -733.634     -160.966  -11.7981   149.168

In [13]:
pyuvws = PyUVW.radec2uvws(o.ra, o.dec, o.jd, o.obslla, o.antxyz, o.bls)

3×78 Matrix{Float64}:
   127.156  1375.15    1612.31    645.441  …  -131.273  152.954    284.227
  2085.19    368.836    279.795   541.411      319.952  -96.4809  -416.433
 -1569.8    -999.813  -1061.35   -733.634     -160.966  -11.7979   149.168

All four produce different answers!  How can we determine which is "best"?  Let's compare...

## Comparing UVW values

To compare UVW values, we can write a simple function that computes
the average of the magnitude of UVW differences between corresponding baseline
vectors (i.e. columns) from two UVW matrices.  We can use this average UVW
difference as a way to quantify how similar two UVW matrices are.

In [14]:
function avguvwdiff(uvw1, uvw2)
    mapreduce(v->hypot(v...), +, eachcol(uvw1-uvw2)) / size(uvw1,2)
end

[
    TopoUVW => avguvwdiff(topouvws, o.uvws),
    GCRSUVW => avguvwdiff(gcrsuvws, o.uvws),
    CASAUVW => avguvwdiff(casauvws, o.uvws),
    PyUVW => avguvwdiff(pyuvws, o.uvws)
]

4-element Vector{Pair{Module, Float64}}:
 UVWExplorer.TopoUVW => 4.318634540521559e-13
 UVWExplorer.GCRSUVW => 0.24365334568963015
 UVWExplorer.CASAUVW => 0.24494298470440581
   UVWExplorer.PyUVW => 0.2426047289735159

The `TopoUVW` calculation produced results that are virtually identical to the
UVW values from the UVH5 file.  This shows that the UVWs in the UVH5 file were
produced not just using the same approach as in `TopoUVW`, but also with
`TopoUVW`'s default values of zero for the Earth orientation parameters `dut1`,
`xp`, and `yp`.  While the three GCRS-based computations produced non-identical
values, they are far closer to each other than to the topocentric-based values.

This shows that there is a fairly significant distinction between the
topocentric and GCRS approaches, but it doesn't really help up determine which
approach is better.  For that we will have to understand more about each
approach.

### The topocentric approach

One way to think about the direction of the phase center is the so called
"observed direction".  This is literally the direction one would "look" from the
observatory's reference position at the observation time to "see" the celestial
position.  Expressing the observed direction as an observed hour angle, `hob`,
and observed declination, `dob`, allows for computing UVW from XYZ by two
single-axis rotations and a permutation of the axes:

1. Rotate the XYZ frame anticlockwise around the Z axis by `(longitude-hob)`,
   producing a (X', U, Z) frame.
2. Rotate that frame anticlockwise around the U
   axis by `-dob`, producing a (W,
   U, V) frame.
3. Permute WUV to UVW.

This approach has a "ground truth" appeal because the computed `W` axis points
in the direction from which the radio wavefront from the phase center appears to
emanate as it passes through the array.  The `W` component of the
topocentric-computed UVW baseline vector therefore is the geometric distance
that the wavefront travels from one antenna of a baseline to the other.

Unfortunately, the `U` and `V` components resulting from this approach have a
few properties that limit their functionality.  The celestial location pointed
to by `W` is nominally in the Celestial Intermediate Reference System (CIRS),
i.e.  after precession and nutation have been applied.  This means the `V`
component nominally points toward the celestial intermediate pole, which varies
over time, rather than a time invariant pole.  Likewise, `U` is similarly offset
from a time invariant "east".  This means that maps made using the
topocentric-computed UVWs from observations on different dates would be slightly
rotated relative to each other.

In theory, the topocentric-computed UVWs could be adjusted through a series of
rotations to align with the ICRS frame, but diurnal aberration would also need
to be accounted for in this exercise, making the process quite complicated.

Another factor that we have been ignoring thus far is refraction, which is why
we have been somewhat loose with the terms "topocentric", which does not include
refraction, and "observed", which does include refraction.  Because refraction
affects the travel path of the wavefront, including refraction in the
calculation of "observed direction" (which "observed" implies anyway) leads to a
`W` component that represents the geometric distance traveled by the wavefront
between antennas of a baseline more accurately than without including
refraction.  The difference in refraction from the "top" (highest elevation) of
the primary beam to the "bottom" (lowest elevation) of the primary beam will
distort the field being observed/imaged, but for the phase center itself the `U`
and `V` components are not significantly affected by the "bent" path of the
wavefront (right?).

### The geocentric approach

Instead of transforming the phase center from the ICRS frame to the
topocentric/observed frame and then rotating the antenna position frame to a
UVW frame, it is possible to avoid the limitations mentioned in the previous
section by using an approach that meets in the middle, i.e. the Geocentric
Celestial Reference System (GCRS).  This approach involves the following steps:

1. Transform the phase center from the ICRS frame to the GCRS frame:
   1. Apply proper motion (generally ignored for extragalactic radio sources)
   2. Apply parallax (generally ignored for extragalactic radio sources)
   3. Apply light defection
   4. Apply aberration (both annual and diurnal)
2. Transform the antenna positions from the ITRF frame to the GCRS frame.  This
   involves the transpose (i.e. inverse) of the *celestial-to-terrestrial*
   rotation matrix such as the one returned by ERFA's `c2t06a` function.
3. Rotate the GCRS frame such that the first axis points to the GCRS phase
   center.  This produces a WUV frame.
4. Permute the WUV frame to a UVW frame.

This approach avoids the issues that `U` and `V` encountered in the
topocentric/observed approach, but the `W` component does not so accurately
represent the geometric distance that the wavefront travels from one antenna to
another.

### Which approach is better?

Which approach is better depends on what you want to do.  For imaging, the `U`
and `V` components are more important the the `W` component.  For delay
calculations, the `W` component is more important than `U` or `V` components.
This leads to the conclusion that the geocentric UVW approach is preferable for
imaging purposes whereas the topocentric/observed UVW approach is preferable for
delay purposes.  However, for delay purposes, computing just the `W` component
of each baseline is more efficient than computing the entire UVW vector.  For
this reason, when the entire UVW baseline vector is needed (e.g. for imaging or
use in a UVH5 file) the geocentric approach is almost always preferable.

## Comparing geocentric UVW implementations

Comparing the three geocentric implementations, `GCRSUVW`, `CASAUVW`, and
`PyUVW`, is a bit tricky due to the way each implementation handles the Earth
orientation parameters `dut1`, `xp`, and `yp`.  While the `radec2uvws` function
accepts these as keyword arguments, the `CASAUVW` and `PyUVW` versions ignore
them since their underlying implementations do not support caller supplied
values for these parameters.  Presumably they use internal mechanisms for
downloading the EOP data from IERS (or elsewhere), but their documentation is a
bit spares on this detail.

We can use the `EarthOrientation` package to obtain EOP values that can be
passed to `GCRSUVW.radec2uvws`:

In [15]:
import EarthOrientation: getΔUT1, getxp, getyp
dut1 = getΔUT1(o.jd)
xp = getxp(o.jd)
yp = getyp(o.jd)
@show dut1 xp yp;

dut1 = -0.0162067860450108
xp = 0.19094269183334256
yp = 0.1968674809638145


Here's what we get when using the EOP values:

In [16]:
gcrseopuvws = GCRSUVW.radec2uvws(o.ra, o.dec, o.jd, o.obslla, o.antxyz, o.bls; dut1, xp, yp)

3×78 Matrix{Float64}:
   127.156  1375.15    1612.31    645.441  …  -131.273  152.954    284.227
  2085.19    368.836    279.795   541.411      319.952  -96.4809  -416.433
 -1569.8    -999.813  -1061.35   -733.634     -160.966  -11.7979   149.168

Let's see what the average UVW difference is between these UVWs and the GCRS UVWs without EOP.

In [17]:
avguvwdiff(gcrsuvws, gcrseopuvws)

0.0014392560422673247

Using EOP values resulted in an average UVW change of almost 1.5 mm compared to not using EOP values.  Let's see how these new values compare with the `CASAUVW` and `PyUVW` values we computed earlier.

In [18]:
[
    "CASAUVW vs GCRSUVW (no EOP)" => avguvwdiff(casauvws, gcrsuvws),
    "CASAUVW vs GCRSUVW (w/ EOP)" => avguvwdiff(casauvws, gcrseopuvws),
    "PyUVW vs GCRSUVW (no EOP)" => avguvwdiff(pyuvws, gcrsuvws),
    "PyUVW vs GCRSUVW (w/ EOP)" => avguvwdiff(pyuvws, gcrseopuvws)
]

4-element Vector{Pair{String, Float64}}:
 "CASAUVW vs GCRSUVW (no EOP)" => 0.001562949981090979
 "CASAUVW vs GCRSUVW (w/ EOP)" => 0.0025850930471770753
   "PyUVW vs GCRSUVW (no EOP)" => 0.001433844971866739
   "PyUVW vs GCRSUVW (w/ EOP)" => 1.4718643270666432e-5

Curiously, when using the EOP values from `EarthOrientation` the comparison with
`CASAUVW` got worse (from 1.6 mm to 2.6 mm), but the comparison with `PyUVW` got
better (from 1.4 mm to 15 µm).  At this point it gets a bit hard to figure out
what exactly is causing these subtle differences since the inner workings of
both the `CASAUVW` and `PyUVW` implementations are rather opaque.  Some
possibilities are:

* Different EOP values (affects the terrestrial-to-celestial transform)
* Different calculations of observer's position/velocity (affects aberration)
* Different calculations of light deflection (GCRSUVW only includes the sun)
* Other differences in astronomical models
* Order of floating point operations (more significant as the differences get
  smaller)

## Fun with minimization

It is interesting to note that practically all of the difference between the
outputs of `GCRSUVW`, `CASAUVW`, and `PyUVW` can be eliminated by tweaking the
EOP values used.  We can use Julia's `Optim` package, a multivariate
optimization package, to find `dut1`, `xp`, and `yp` values that minimize the
difference.  We just need to define a `residual` function for `Optim` to
minimize.

In [19]:
using Optim

# `x` will be a vector containing candidate [dut1, xp, yp] values
# `o` will be a named tuple containing constant parameters for `radec2uvws`
# `ref` will be the "target" UVW array
function residual(x, o, ref)
    dut1, xp, yp = x
    gcrsuvw = GCRSUVW.radec2uvws(o.ra, o.dec, o.jd, o.obslla, o.antxyz, o.bls; dut1, xp, yp)
    avguvwdiff(gcrsuvw, ref)
end;

### Minimize residuals for `casauvws`

In [20]:
casaresults = optimize(x->residual(x, o, casauvws), [dut1, xp, yp], Optim.Options(g_tol=1e-16))

 * Status: success

 * Candidate solution
    Final objective value:     4.763398e-12

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-16

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    181
    f(x) calls:    382


### Minimize residuals for `pyuvws`

In [21]:
pyresults=optimize(x->residual(x, o, pyuvws), [dut1, xp, yp], Optim.Options(g_tol=1e-16))

 * Status: success

 * Candidate solution
    Final objective value:     3.104862e-13

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-16

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    174
    f(x) calls:    361


Both optimizations converged to tiny residuals ("Final objective value").

Here are the `dut1`, `xp`, and `yp` values from `EarthOrientation` and the
`CASAUVW` and `PyUVW` minimizations.

In [22]:
dut1c, xpc, ypc = Optim.minimizer(casaresults)
dut1p, xpp, ypp = Optim.minimizer(pyresults)

@show dut1 dut1c dut1p; println()
@show xp xpc xpp; println()
@show yp ypc ypp;

dut1 = -0.0162067860450108
dut1c = -0.008011131562502705
dut1p = -0.016010515288670178

xp = 0.19094269183334256
xpc = -0.0013006328319094655
xpp = 0.19230044654781023

yp = 0.1968674809638145
ypc = -0.4095296116832459
ypp = 0.1955412449801455


The `PyUVW` minimization resulted in EOP values very close to the
`EarthOrientation` values, well within the differences between various IERS EOP
products.  This suggests that `PyUVW` and `GCRSUVW` are very similar
implementations.

Despite producing a very small residual, the `CASAUVW` minimization resulted in
EOP values that are not so close to the `EarthOrientation` values, far outside
the differences between various IERS EOP products.  This suggests that CASA's
UVW calculation is somewhat different from those of the `GCRSUVW` and `PyUVW`
implementations.

Just for fun, let's take a look at the UVW differences when using these
minimizing EOP values to see how well the minimization process worked.

In [23]:
casauvws - GCRSUVW.radec2uvws(o.ra, o.dec, o.jd, o.obslla, o.antxyz, o.bls; dut1=dut1c, xp=xpc, yp=ypc)

3×78 Matrix{Float64}:
 7.95808e-12  4.54747e-12  5.45697e-12  …  5.68434e-14  -7.95808e-13
 3.18323e-12  3.0127e-12   3.41061e-12     1.56319e-13  -1.13687e-13
 7.04858e-12  9.89075e-12  1.15961e-11     6.50147e-13   5.40012e-13

In [24]:
pyuvws - GCRSUVW.radec2uvws(o.ra, o.dec, o.jd, o.obslla, o.antxyz, o.bls; dut1=dut1p, xp=xpp, yp=ypp)

3×78 Matrix{Float64}:
 -1.13687e-13  -4.54747e-13  -2.27374e-13  …  0.0          -1.13687e-13
 -4.54747e-13   1.13687e-13   0.0             0.0           5.68434e-14
  4.54747e-13   3.41061e-13   4.54747e-13     3.55271e-15   0.0

## Where's my observatory?

One other detail that make comparing opaque implementations of UVW calculations
tricky is that some implementations, such as both `CASA` and `astropy` have
built in positions for known observatories that are inconsistent with each
other!  Let's look at the ITRF coordinates for the VLA from `CASA`, `astropy`,
and the example UVH5 file to see how different they are.

The `CASAUVW` and `PyUVW` modules provide a `vla_location` function that return
the built-in location of the VLA for `CASA` and `astropy`, resp.  We will use
the `gd2gc` function from `ERFA` to convert `o.obslla` to ITRF using the WGS84
ellipsoid.

In [42]:
import ERFA: gd2gc, WGS84
casavla = CASAUVW.vla_location()
pyvla = PyUVW.vla_location()
uvh5vla = gd2gc(WGS84, deg2rad(o.obslla.lon), deg2rad(o.obslla.lat), o.obslla.alt)
[
    "casa-astropy" => (casavla-pyvla, hypot((casavla-pyvla)...)),
    "casa-uvh5" => (casavla-uvh5vla, hypot((casavla-uvh5vla)...)),
    "astrppy-uvh5" => (pyvla-uvh5vla, hypot((pyvla-uvh5vla)...))
]

3-element Vector{Pair{String, Tuple{Vector{Float64}, Float64}}}:
 "casa-astropy" => ([-0.9630800811573863, 12.408692349679768, 0.7931435378268361], 12.471256782831462)
    "casa-uvh5" => ([-42.829292317153886, 34.70993820298463, 13.481200681068003], 56.75271678295636)
 "astrppy-uvh5" => ([-41.8662122359965, 22.301245853304863, 12.688057143241167], 49.103076152815554)

The `CASA` and `astropy` VLA locations are 12 meters apart.  The VLA location in
the UVH5 file is ~50 meters away from the other two locations.  This is
primarily because the latitude and longitude in the UVH5 file are consistent
with sexagesimal representations that have only arcsecond precision,
specifically latitude `+34:04:43` and longitude `-107:37:04`.

In [32]:
(34 + (4 + 43/60)/60, -(107 + (37 + 4/60)/60)) == (o.obslla.lat, o.obslla.lon)

true

Using different observatory positions with the same `radec2uvws` function will
show how much these various positions affect the resultant UVW values.  For we
need a function to convert the ITRF locations to LLA format.

In [45]:
import ERFA: gc2gd

function gc2lla(xyz)
    lon, lat, alt = gc2gd(WGS84, xyz)
    (lat=rad2deg(lat), lon=rad2deg(lon), alt=alt)
end

casalla = gc2lla(casavla)
pylla = gc2lla(pyvla)

@show o.obslla casalla pylla;

o.obslla = (lat = 34.07861111111111, lon = -107.61777777777777, alt = 2124.0)
casalla = (lat = 34.07881333755048, lon = -107.61833367504566, alt = 2114.890229039043)
pylla = (lat = 34.07874916666667, lon = -107.61828305555552, alt = 2123.9999999997867)


The repeating digits at the end of `pylla`'s latitude and longitude suggest that
`astropy`'s VLA location also came from a sexagesimal representation,
specifically latitude `+34:04:43.497` and longitude `-107:37:05.819`, both to
milliarcsecond precision.

Let's see how much difference this makes in the resultant UVWs.

In [39]:
avguvwdiff(CASAUVW.radec2uvws(o.ra, o.dec, o.jd, casalla, o.antxyz, o.bls),
           CASAUVW.radec2uvws(o.ra, o.dec, o.jd,   pylla, o.antxyz, o.bls))

2.7786519954576024e-8

In [40]:
avguvwdiff(CASAUVW.radec2uvws(o.ra, o.dec, o.jd,  casalla, o.antxyz, o.bls),
           CASAUVW.radec2uvws(o.ra, o.dec, o.jd, o.obslla, o.antxyz, o.bls))

3.831621858235637e-9

In [41]:
avguvwdiff(CASAUVW.radec2uvws(o.ra, o.dec, o.jd,    pylla, o.antxyz, o.bls),
           CASAUVW.radec2uvws(o.ra, o.dec, o.jd, o.obslla, o.antxyz, o.bls))


2.4622514578984983e-8

The different VLA locations lead to UVW differences on the order of 10 nm.
While this is a negligible difference, it is perhaps surprising that it's not
zero (or closer to zero) since the reference location "cancels out" when
subtracting baseline positions.  The different locations lead to different
observatory positions and velocities, which lead to slightly different amounts
of aberration (and to a lesser extent light deflection).